source: https://openai.github.io/openai-agents-python/context/
# Context management

1. Context available locally to your code: this is data and dependencies you might need when tool functions run, during callbacks like on_handoff, in lifecycle hooks, etc.
2. Context available to LLMs: this is data the LLM sees when generating a response.

## Local context
Represented via the RunContextWrapper class and the context property within it. The way this works is:

- You create any Python object you want. A common pattern is to use a dataclass or a Pydantic object.
- You pass that object to the various run methods (e.g. Runner.run(..., **context=whatever**)).
- All your tool calls, lifecycle hooks etc will be passed a wrapper object, RunContextWrapper[T], where T = context object type accessible via wrapper.context.

The most important thing to be aware of: every agent, tool function, lifecycle etc for a given agent run **must use the same type of context.**

You can use it for:

- Contextual data for your run (e.g. things like a username/uid or other information about the user)
- Dependencies (e.g. logger objects, data fetchers, etc)
- Helper functions

Note
The context object is not sent to the LLM. It is purely a local object that you can read from, write to and call methods on it.


In [4]:
import nest_asyncio
nest_asyncio.apply()

from dataclasses import dataclass
from agents import Agent, RunContextWrapper, Runner, function_tool

@dataclass
class ProductInfo:  
    product_name: str
    product_id: int

@function_tool
async def fetch_product_price(wrapper: RunContextWrapper[ProductInfo]) -> str:  
    """Fetch the price of the product. Call this function to get product's price information."""
    return f"The product {wrapper.context.product_name} costs $99.99"


product_info = ProductInfo(product_name="SuperWidget", product_id=456)

agent = Agent[ProductInfo](  
    name="ProductAssistant",
    tools=[fetch_product_price],
)

# In a Jupyter notebook, you need to use 'await' for async functions,
# unless you use an event loop runner like asyncio.run (not recommended in notebooks),
# or use IPython's built-in 'await' support (which is available in modern Jupyter).
# So, you should keep 'await' here:
result = await Runner.run(  
    starting_agent=agent,
    input="What is the price of the product?",
    context=product_info,
)

print(result.final_output)  
# The product SuperWidget costs $99.99.

The price of the product, SuperWidget, is $99.99.


# Agent/LLM context

To make new data available to an LLM, you must add it to the conversation history. Main ways to do this:

- Add it to Agent instructions (system prompt), either as a static string or a dynamic function.
- Include it in the input when calling Runner.run.
- Expose it via function tools for on-demand access.
- Use retrieval or web search tools to fetch relevant data as needed.

In [ ]:
# You can experiment with different ways to expose data to the LLM here:

